## **Multi-Class Image Classification**

- **Data Preparation with OpenCV and NumPy**

In [ ]:
import numpy as np
import pickle
import os
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

def load_cifar10_data(data_dir):
    train_data = []
    train_labels = []
    for batch in range(1, 6):
        batch_data = unpickle(os.path.join(data_dir, f'data_batch_{batch}'))
        train_data.append(batch_data[b'data'])
        train_labels.extend(batch_data[b'labels'])
    
    train_data = np.concatenate(train_data)
    train_data = train_data.reshape((50000, 3, 32, 32)).transpose(0, 2, 3, 1)
    train_labels = np.array(train_labels)
    
    test_data = unpickle(os.path.join(data_dir, 'test_batch'))
    test_images = test_data[b'data'].reshape((10000, 3, 32, 32)).transpose(0, 2, 3, 1)
    test_labels = np.array(test_data[b'labels'])
    
    return (train_data, train_labels), (test_images, test_labels)

data_dir = 'data'
(train_images, train_labels), (test_images, test_labels) = load_cifar10_data(data_dir)


train_images = train_images.astype('float32') / 255.0
test_images = test_images.astype('float32') / 255.0


train_labels = to_categorical(train_labels, 10)
test_labels = to_categorical(test_labels, 10)


datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

datagen.fit(train_images)

print('Training data shape:', train_images.shape)


- **Extracting Local Features of the Images**

In [ ]:
import cv2
from skimage.feature import hog, local_binary_pattern

def extract_hog_features(images):
    hog_features = []
    for img in images:
        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        hog_feature = hog(gray_img, block_norm='L2-Hys')
        hog_features.append(hog_feature)
    return np.array(hog_features)

def extract_lbp_features(images):
    lbp_features = []
    for img in images:
        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        lbp_feature = local_binary_pattern(gray_img, P=8, R=1, method='uniform')
        lbp_feature = np.histogram(lbp_feature, bins=np.arange(257), density=True)[0]
        lbp_features.append(lbp_feature)
    return np.array(lbp_features)

# Extract HOG features
train_hog_features = extract_hog_features(train_images)
test_hog_features = extract_hog_features(test_images)

# Extract LBP features
train_lbp_features = extract_lbp_features(train_images)
test_lbp_features = extract_lbp_features(test_images)



- **Training Model on Dataset**

In [ ]:
# Using VGG16 for deep feature extraction
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model

# Load pre-trained VGG16 model + higher level layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Extract features
train_cnn_features = base_model.predict(train_images)
test_cnn_features = base_model.predict(test_images)

# Flatten the features for SVM
train_cnn_features = train_cnn_features.reshape((train_cnn_features.shape[0], -1))
test_cnn_features = test_cnn_features.reshape((test_cnn_features.shape[0], -1))

- **Applying PCA**

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=256)
train_pca = pca.fit_transform(train_cnn_features)
test_pca = pca.transform(test_cnn_features)

- **Evaluating Model**

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# SVM Classifier with RBF kernel
svm_rbf = SVC(kernel='rbf', C=1, gamma='scale')
svm_rbf.fit(train_pca, train_labels.argmax(axis=1))

# Predictions
test_pred = svm_rbf.predict(test_pca)

# Evaluation
accuracy = accuracy_score(test_labels.argmax(axis=1), test_pred)
precision = precision_score(test_labels.argmax(axis=1), test_pred, average='weighted')
recall = recall_score(test_labels.argmax(axis=1), test_pred, average='weighted')
f1 = f1_score(test_labels.argmax(axis=1), test_pred, average='weighted')

print(f"SVM Test accuracy: {accuracy * 100:.2f}%")
print(f"SVM Precision: {precision:.2f}")
print(f"SVM Recall: {recall:.2f}")
print(f"SVM F1-Score: {f1:.2f}")


- **Saving Trained Model**

In [ ]:
import pickle

# Assume `svm_rbf`, `pca` are your trained SVM model and PCA object
with open('svm_model.pkl', 'wb') as f:
    pickle.dump(svm_rbf, f)

with open('pca_object.pkl', 'wb') as f:
    pickle.dump(pca, f)